In [1]:
from google.colab import files
uploaded=files.upload()

Saving ARIALMT.ttf to ARIALMT.ttf
Saving Calibri.ttf to Calibri.ttf


In [2]:
fonts=[]
for font in uploaded:
  fonts.append(font)
fonts

[u'ARIALMT.ttf', u'Calibri.ttf']

In [3]:
out_charset="ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789"
print len(out_charset)
b=dict(zip(range(len(out_charset)),out_charset))
print b
a=dict(zip(out_charset,range(len(out_charset))))
print a

62
{0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'I', 9: 'J', 10: 'K', 11: 'L', 12: 'M', 13: 'N', 14: 'O', 15: 'P', 16: 'Q', 17: 'R', 18: 'S', 19: 'T', 20: 'U', 21: 'V', 22: 'W', 23: 'X', 24: 'Y', 25: 'Z', 26: 'a', 27: 'b', 28: 'c', 29: 'd', 30: 'e', 31: 'f', 32: 'g', 33: 'h', 34: 'i', 35: 'j', 36: 'k', 37: 'l', 38: 'm', 39: 'n', 40: 'o', 41: 'p', 42: 'q', 43: 'r', 44: 's', 45: 't', 46: 'u', 47: 'v', 48: 'w', 49: 'x', 50: 'y', 51: 'z', 52: '0', 53: '1', 54: '2', 55: '3', 56: '4', 57: '5', 58: '6', 59: '7', 60: '8', 61: '9'}
{'1': 53, '0': 52, '3': 55, '2': 54, '5': 57, '4': 56, '7': 59, '6': 58, '9': 61, '8': 60, 'A': 0, 'C': 2, 'B': 1, 'E': 4, 'D': 3, 'G': 6, 'F': 5, 'I': 8, 'H': 7, 'K': 10, 'J': 9, 'M': 12, 'L': 11, 'O': 14, 'N': 13, 'Q': 16, 'P': 15, 'S': 18, 'R': 17, 'U': 20, 'T': 19, 'W': 22, 'V': 21, 'Y': 24, 'X': 23, 'Z': 25, 'a': 26, 'c': 28, 'b': 27, 'e': 30, 'd': 29, 'g': 32, 'f': 31, 'i': 34, 'h': 33, 'k': 36, 'j': 35, 'm': 38, 'l': 37, 'o': 40, 'n': 39

In [0]:
def function(sequence):
    l=[]
    for char in sequence:
        if char not in out_charset:
            return -1
        l.append(a[char])
    return l

In [0]:
import cv2
from PIL import Image
import numpy as np
from PIL import ImageFont
from PIL import ImageDraw
import os
import shutil

In [0]:
data=[]
labels=[]
length=[]

def GenerateCharacters():  
    k = 1
    for filename in fonts:  
        font_resource_file = filename   
        for word in words:
            if len(word)>12:
                continue
            l=function(word)
            if l==-1:
              continue
            for font_size in font_sizes:
                font = ImageFont.truetype(font_resource_file, font_size)
                TEXT=word
                txt = TEXT
                (width, height) = font.getsize(txt)

                #New Image
                FOREGROUND = (255)
                background = Image.new('L', (width, height), color = 0)

                draw = ImageDraw.Draw(background)
                draw.text((0,0), txt, font = font, fill=FOREGROUND)

                w,h = background.size
                W = int(round(w/float(h)*22))
                req_im = np.asarray(background.resize((W, 22)))
                if(W<200):
                    req_im = np.hstack([req_im, np.zeros((22,200-W),dtype='uint8')])
                else:
                    req_im=Image.fromarray(req_im)
                    req_im=req_im.resize((200,22))
                    req_im=np.asarray(req_im)
                word_image=req_im
#                 outpath=out_dir+word
#                 if not os.path.exists(outpath):
#                     os.makedirs(outpath)
#                 file_name = os.path.join(outpath,str(k)+'.jpg')  
#                 cv2.imwrite(file_name,word_image)
                data.append(word_image)
                l=np.array(l)
#                 pad=np.full((1,12-l.shape[0]),-1)
#                 l=np.append(l,pad)
                labels.append(l)
                length.append(len(word))
                k = k + 1   
    return

In [7]:
from google.colab import files
uploaded=files.upload()

Saving sample_words.txt to sample_words.txt


In [0]:
font_sizes=[22]
background_color=255
fillcolor=0
with open('sample_words.txt','rb') as f:
    word=f.read().split()
words=word
GenerateCharacters()

In [11]:
print data[0].shape
print labels[2]
print len(data)
print len(labels)
print len(length)
print length[0]

(22, 200)
[46 39 29 30 43 44 30 37 37]
32062
32062
32062
6


In [12]:
for label in labels[:5]:
  print label.shape

(6,)
(7,)
(9,)
(8,)
(9,)


In [0]:
def sparse_tuples(x):
  x_ix = []
  x_val = []
  for batch_i, batch in enumerate(x):
    for time, val in enumerate(batch):
      x_ix.append([batch_i, time])
      x_val.append(val)
  x_shape = [len(x), np.asarray(x_ix).max(0)[1]+1]
  x_ix = tf.constant(x_ix, tf.int64)
  x_val = tf.constant(x_val, tf.int32)
  x_shape = tf.constant(x_shape, tf.int64)

  return tf.SparseTensorValue(x_ix, x_val, x_shape)

In [0]:
data=np.array(data,dtype="float32")/255.0
data=data.reshape(data.shape[0],data.shape[1],data.shape[2],1)
labels=np.array(labels)
length=np.array(length)

In [15]:
print data.dtype
print data.shape
print labels.shape
print labels
print length
print length.shape
print max(length)

float32
(32062, 22, 200, 1)
(32062,)
[array([40, 46, 45, 27, 34, 29]) array([44, 41, 30, 39, 29, 30, 43])
 array([46, 39, 29, 30, 43, 44, 30, 37, 37]) ...
 array([40, 46, 45, 31, 40, 46, 32, 33, 45])
 array([46, 39, 29, 30, 43, 38, 26, 39, 39, 30, 29])
 array([40, 46, 45, 38, 26, 39, 39, 30, 29])]
[ 6  7  9 ...  9 11  9]
(32062,)
12


In [0]:
height=22
width=200
depth=1
classes=len(out_charset)

In [17]:
import tensorflow as tf
print tf.__version__

batch_size=32
print data.shape
print labels.shape
train_data=data[:int(data.shape[0]*0.7)]
train_labels=labels[:int(labels.shape[0]*0.7)]
train_seq_len=length[:int(length.shape[0]*0.7)]
val_data=data[int(data.shape[0]*0.7):int(data.shape[0]*0.8)]
val_labels=labels[int(labels.shape[0]*0.7):int(labels.shape[0]*0.8)]
val_seq_len=length[int(length.shape[0]*0.7):int(length.shape[0]*0.8)]
test_data=data[int(data.shape[0]*0.8):]
test_labels=labels[int(labels.shape[0]*0.8):]
test_seq_len=length[int(length.shape[0]*0.8):]
print train_seq_len[1]
print val_seq_len[0]
print test_seq_len[0]

1.9.0-rc1
(32062, 22, 200, 1)
(32062,)
7
11
6


In [0]:
# sparse_train_labels=sparse_tuples(train_labels)
# sparse_val_labels=sparse_tuples(val_labels)

In [0]:
# dx_train= tf.data.Dataset.from_tensor_slices(train_data)
# dy_train= tf.data.Dataset.from_tensor_slices(train_labels)
# dz_train=tf.data.Dataset.from_tensor_slices(train_seq_len)
# train_dataset = tf.data.Dataset.zip((dx_train, dy_train,dz_train)).shuffle(500).batch(batch_size)

In [0]:
# dx_val= tf.data.Dataset.from_tensor_slices(val_data)
# dy_val= tf.data.Dataset.from_tensor_slices(val_labels)
# dz_val=tf.data.Dataset.from_tensor_slices(val_seq_len)
# val_dataset = tf.data.Dataset.zip((dx_val, dy_val,dz_val)).shuffle(500).batch(batch_size)

In [0]:
# iterator = tf.data.Iterator.from_structure(train_dataset.output_types,train_dataset.output_shapes)
# next_element = iterator.get_next()

In [0]:
# training_init_op = iterator.make_initializer(train_dataset)
# validation_init_op = iterator.make_initializer(val_dataset)

In [0]:
# conv1_trim = tf.constant( 2 * (3 // 2),
#                                   dtype=tf.int32,
#                                   name='conv1_trim')
# one = tf.constant(1, dtype=tf.int32, name='one')
# two = tf.constant(2, dtype=tf.int32, name='two')
# after_conv1 = tf.subtract( widths, conv1_trim)
# after_pool2 = tf.floor_div( after_conv1, two )
# after_pool4 = tf.subtract(after_pool2, one)
# after_pool6 = tf.subtract(after_pool4, one) 
# after_pool8 = after_pool6

In [0]:
# # input_data=tf.placeholder(tf.float32,[None, height, width, depth])
# # print input_layer.shape

# def model(input_layer):
#   conv1=tf.layers.conv2d(inputs=input_layer,
#         filters=64,
#         kernel_size=[3, 3],
#         strides=[1,1],
#         padding="same",
#         activation=tf.nn.relu)
#   print "Conv1",conv1.shape
#   bn1=tf.layers.batch_normalization(inputs=conv1,axis=-1)
#   print "BN1",bn1.shape
#   pool1=tf.layers.max_pooling2d(inputs=bn1,pool_size=[2,2],strides=[2,2])
#   print "Pool1",pool1.shape
#   conv2=tf.layers.conv2d(inputs=pool1,
#         filters=128,
#         kernel_size=[3, 3],
#         strides=[1,1],
#         padding="same",
#         activation=tf.nn.relu)
#   print "Conv2",conv2.shape
#   bn2=tf.layers.batch_normalization(inputs=conv2,axis=-1)
#   print "BN2",bn2.shape
#   pool2=tf.layers.max_pooling2d(inputs=bn2,pool_size=[2,2],strides=[2,2])
#   print "Pool2",pool2.shape
#   conv3=tf.layers.conv2d(inputs=pool2,
#         filters=256,
#         kernel_size=[3, 3],
#         strides=[1,1],
#         padding="same",
#         activation=tf.nn.relu)
#   print "Conv3",conv3.shape
#   bn3=tf.layers.batch_normalization(inputs=conv3,axis=-1)
#   print "BN3",bn3.shape
#   pool3=tf.layers.max_pooling2d(inputs=bn3,pool_size=[2,2],strides=[2,2])
#   print "Pool3",pool3.shape
#   conv4=tf.layers.conv2d(inputs=pool3,
#         filters=512,
#         kernel_size=[3, 3],
#         strides=[1,1],
#         padding="same",
#         activation=tf.nn.relu)
#   print "Conv4",conv4.shape
#   bn4=tf.layers.batch_normalization(inputs=conv4,axis=-1)
#   print "BN4",bn4.shape
#   pool4=tf.layers.max_pooling2d(inputs=bn4,pool_size=[2,2],strides=[2,2])
#   print "Pool4",pool4.shape
#   features = tf.squeeze(pool4, axis=1, name='features')
#   print "CNN features",features.shape

#   rnn_ = tf.transpose(features, perm=[1, 0, 2], name='time_major')
#   print "RNN_seq",rnn_.shape



#   weight_initializer = tf.truncated_normal_initializer(stddev=0.01)
#   with tf.variable_scope('forward'):
#       cell_fw = tf.contrib.rnn.LSTMCell(512,initializer=weight_initializer)
#   with tf.variable_scope('backward'):
#       cell_bw = tf.contrib.rnn.LSTMCell(512,initializer=weight_initializer) 
#   with tf.variable_scope('Bdrnn',reuse=tf.AUTO_REUSE):
#     rnn_output,_ = tf.nn.bidirectional_dynamic_rnn(cell_fw, cell_bw, rnn_,
#       sequence_length=sequence_length,
#       time_major=True,
#     dtype=tf.float32)

#   rnn_output_stack = tf.concat(rnn_output,2,name='output_stack')
#   logit_activation = tf.nn.relu
#   weight_initializer = tf.contrib.layers.variance_scaling_initializer()
#   bias_initializer = tf.constant_initializer(value=0.0)
#   rnn_logits = tf.layers.dense( rnn_output_stack, classes+1, 
#                                         activation=logit_activation,
#                                         kernel_initializer=weight_initializer,
#                                         bias_initializer=bias_initializer)
#   return rnn_logits

In [0]:

# loss = tf.nn.ctc_loss(labelss,logit_output,sequence_length=sequence_length,time_major=True )
# print loss
# total_loss = tf.reduce_mean(loss)
# print total_loss

In [0]:
# optimizer = tf.train.AdamOptimizer().minimize(loss)
# # get accuracy
# prediction = tf.argmax(logit_output, 1)
# equality = tf.equal(prediction, tf.argmax(next_element[1], 1))
# accuracy = tf.reduce_mean(tf.cast(equality, tf.float32))
# init_op = tf.global_variables_initializer()

In [0]:
def batch_data(data,batch_num,total_batches,bs):
  if batch_num==total_batches-1:
    return data[batch_num*bs:]
  else:
    return data[batch_num*bs:(batch_num+1)*bs]
  

In [28]:
import math
d=np.arange(0,10)
print batch_data(d,2,4,3)
tb=int(math.ceil(10/3.0))
tb

[6 7 8]


4

In [29]:
FIRST_INDEX = ord('a') - 1
epochs = 5
batch_size=1
num_train_examples=train_data.shape[0]
num_valid_examples=val_data.shape[0]
total_train_batches=int(math.ceil(num_train_examples/batch_size*1.0))
total_val_batches=int(math.ceil(num_valid_examples/batch_size*1.0))
print num_train_examples
print num_valid_examples
print total_train_batches
print total_val_batches

22443
3206
701
100


In [0]:
def convert(ar):
  out=""
  for index in ar:
      out+=b[index]
  return out
    

In [0]:
f=np.array([2,1,2,4,5,62])

In [0]:
def run_ctc():
  graph = tf.Graph()
  with graph.as_default():
    input_data=tf.placeholder(tf.float32,[None, height, width, depth])
    labelss=tf.sparse_placeholder(tf.int32)
    sequence_length=tf.placeholder(tf.int32, [None])
    
    conv1=tf.layers.conv2d(inputs=input_data,
        filters=64,
        kernel_size=[3, 3],
        strides=[1,1],
        padding="same",
        activation=tf.nn.relu)
    print "Conv1",conv1.shape
    bn1=tf.layers.batch_normalization(inputs=conv1,axis=-1)
    print "BN1",bn1.shape
    pool1=tf.layers.max_pooling2d(inputs=bn1,pool_size=[2,2],strides=[2,2])
    print "Pool1",pool1.shape
    conv2=tf.layers.conv2d(inputs=pool1,
        filters=128,
        kernel_size=[3, 3],
        strides=[1,1],
        padding="same",
        activation=tf.nn.relu)
    print "Conv2",conv2.shape
    bn2=tf.layers.batch_normalization(inputs=conv2,axis=-1)
    print "BN2",bn2.shape
    pool2=tf.layers.max_pooling2d(inputs=bn2,pool_size=[2,2],strides=[2,2])
    print "Pool2",pool2.shape
    conv3=tf.layers.conv2d(inputs=pool2,
        filters=256,
        kernel_size=[3, 3],
        strides=[1,1],
        padding="same",
        activation=tf.nn.relu)
    print "Conv3",conv3.shape
    bn3=tf.layers.batch_normalization(inputs=conv3,axis=-1)
    print "BN3",bn3.shape
    pool3=tf.layers.max_pooling2d(inputs=bn3,pool_size=[2,2],strides=[2,2])
    print "Pool3",pool3.shape
    conv4=tf.layers.conv2d(inputs=pool3,
        filters=512,
        kernel_size=[3, 3],
        strides=[1,1],
        padding="same",
        activation=tf.nn.relu)
    print "Conv4",conv4.shape
    bn4=tf.layers.batch_normalization(inputs=conv4,axis=-1)
    print "BN4",bn4.shape
    pool4=tf.layers.max_pooling2d(inputs=bn4,pool_size=[2,2],strides=[2,2])
    print "Pool4",pool4.shape
    features = tf.squeeze(pool4, axis=1, name='features')
    print "CNN features",features.shape

    rnn_ = tf.transpose(features, perm=[1, 0, 2], name='time_major')
    print "RNN_seq",rnn_.shape



    weight_initializer = tf.truncated_normal_initializer(stddev=0.01)
    with tf.variable_scope('forward'):
        cell_fw = tf.contrib.rnn.LSTMCell(512,initializer=weight_initializer)
    with tf.variable_scope('backward'):
        cell_bw = tf.contrib.rnn.LSTMCell(512,initializer=weight_initializer) 
    with tf.variable_scope('Bdrnn',reuse=tf.AUTO_REUSE):
      rnn_output,_ = tf.nn.bidirectional_dynamic_rnn(cell_fw, cell_bw, rnn_,
        sequence_length=sequence_length,
        time_major=True,
      dtype=tf.float32)

    rnn_output_stack = tf.concat(rnn_output,2,name='output_stack')
    logit_activation = tf.nn.relu6
    weight_initializer = tf.contrib.layers.variance_scaling_initializer()
    bias_initializer = tf.constant_initializer(value=0.0)
    logit_output = tf.layers.dense( rnn_output_stack, classes+1, 
                                        activation=logit_activation,
                                        kernel_initializer=weight_initializer,
                                        bias_initializer=bias_initializer)

    loss = tf.nn.ctc_loss(labelss,logit_output,sequence_length,time_major=True )
    total_loss = tf.reduce_mean(loss)
    optimizer = tf.train.AdamOptimizer(learning_rate=1e-3).minimize(total_loss)      
    decoded, log_prob = tf.nn.ctc_beam_search_decoder(logit_output, sequence_length,beam_width=4,merge_repeated=False)
    ler = tf.reduce_mean(tf.edit_distance(tf.cast(decoded[0], tf.int32),labelss))
    
    
    with tf.Session(graph=graph) as sess:
      init=tf.global_variables_initializer()
      init_l=tf.local_variables_initializer()
      sess.run(init)
      sess.run(init_l)
      for i in range(epochs):
        train_cost = train_ler = 0
        print("Epoch: {}").format(i)
        for batch_no in range(total_train_batches):
          
          feed = {input_data: batch_data(train_data,batch_no,total_train_batches,batch_size),
                 labelss:sess.run(sparse_tuples(batch_data(train_labels,batch_no,total_train_batches,batch_size))),
                 sequence_length: batch_data(train_seq_len,batch_no,total_train_batches,batch_size)}
 
          da,l,seq=sess.run([input_data,labelss,sequence_length],feed_dict=feed)
          logits=sess.run(logit_output,feed_dict=feed)
#           print "Logits",logits.shape
#           print da.shape
#           print seq.shape

      #     prediction=tf.argmax(tf.nn.softmax(logit_output),axis=2)
#           original=convert(train_labels[batch_no])
#           print "Original Label",original
          batch_cost,_ = sess.run([total_loss,optimizer],feed_dict=feed)
          train_cost += batch_cost*batch_size
          train_ler += sess.run(ler, feed_dict=feed)*batch_size
#           print "Batch_loss",batch_cost
#           d = sess.run(decoded[0], feed_dict=feed)
#           str_decoded = convert(d[1])
#           print "Decoded Label",str_decoded
        train_cost /= num_train_examples
        train_ler /= num_train_examples

        for batch_no in range(total_val_batches):
          val_feed = {input_data: batch_data(val_data,batch_no,total_val_batches,batch_size),
                      labelss: sess.run(sparse_tuples(batch_data(val_labels,batch_no,total_val_batches,batch_size))),
                      sequence_length: batch_data(val_seq_len,batch_no,total_val_batches,batch_size)}

          val_cost,val_ler = sess.run([total_loss, ler], feed_dict=val_feed)

          log = "Epoch {}/{}, train_cost = {:.3f}, train_ler = {:.3f}, val_cost = {:.3f}, val_ler = {:.3f}"
          print(log.format(i+1, epochs, train_cost, train_ler,val_cost, val_ler))
          original=convert(val_labels[batch_no])
          print "Original Label",original
          d = sess.run(decoded[0], feed_dict=val_feed)
          str_decoded = convert(d[1])
          print "Decoded Label",str_decoded
          

         

  

  

In [0]:
if __name__ == '__main__':
  run_ctc()

Conv1 (?, 22, 200, 64)
BN1 (?, 22, 200, 64)
Pool1 (?, 11, 100, 64)
Conv2 (?, 11, 100, 128)
BN2 (?, 11, 100, 128)
Pool2 (?, 5, 50, 128)
Conv3 (?, 5, 50, 256)
BN3 (?, 5, 50, 256)
Pool3 (?, 2, 25, 256)
Conv4 (?, 2, 25, 512)
BN4 (?, 2, 25, 512)
Pool4 (?, 1, 12, 512)
CNN features (?, 12, 512)
RNN_seq (12, ?, 512)
Epoch: 0


In [0]:
# with tf.Session() as sess:
#     indices=sparse_labels[0]
#     values=sparse_labels[1]
#     shape=sparse_labels[2]
#     sp = tf.SparseTensor(indices=indices, values=values, dense_shape=shape)
#     sp_value = sp.eval(session=sess)
#     init = tf.global_variables_initializer()
#     sess.run(init) 
#     num_epochs=5
#     batch_size=2**5
#     total_batches=num_training_examples/batch_size
#     for epoch in range(num_epochs):
#         print sess.run(total_loss,feed_dict={input_data:data,labelss:sp_value})

In [53]:
# num_epochs=5
# num_examples=train_data.shape[0]
# batch_size=32
# num_batches_per_epoch=int(num_examples/batch_size)
# FIRST_INDEX = ord('a') - 1 # 0 is reserved to space

# graph = tf.Graph()
# with graph.as_default():  
#   input_data=tf.placeholder(tf.float32,[None, height, width, depth])
#   labelss=tf.sparse_placeholder(tf.int32)
#   sequence_length=tf.placeholder(tf.int32, [None])

#   logit_output=model(input_data)

#   loss = tf.nn.ctc_loss(labelss,logit_output,sequence_length=sequence_length,time_major=True )
#   print loss
#   cost = tf.reduce_mean(loss)
#   print cost
#   optimizer = tf.train.AdamOptimizer().minimize(cost)
#   decoded, log_prob = tf.nn.ctc_greedy_decoder(logit_output, sequence_length)
#   ler = tf.reduce_mean(tf.edit_distance(tf.cast(decoded[0], tf.int32),labelss))

# with tf.Session(graph=graph) as session:
#   tf.global_variables_initializer().run()
#   for curr_epoch in range(num_epochs):
#     train_cost = train_ler = 0
#     for batch in range(num_batches_per_epoch):
#       feed = {input_data: train_data,
#               labelss: sparse_train_labels,
#               sequence_length:train_seq_len}
#       batch_cost, _ = session.run([cost, optimizer], feed)
#       train_cost += batch_cost*batch_size
#       train_ler += session.run(ler, feed_dict=feed)*batch_size

#     train_cost /= num_examples
#     train_ler /= num_examples

#     val_feed = {input_data: val_data,
#                 labelss: sparse_val_labels,
#                 sequence_length: val_seq_len}

#     val_cost = session.run([cost, ler], feed_dict=val_feed)

#     log = "Epoch {}/{}, train_cost = {:.3f}, train_ler = {:.3f}, val_cost = {:.3f}, val_ler = {:.3f}"
#     print(log.format(curr_epoch+1, num_epochs, train_cost, train_ler,val_cost, val_ler))

#   d = session.run(decoded[0], feed_dict=feed)
#   str_decoded = ''.join([chr(x) for x in np.asarray(d[1]) + FIRST_INDEX])
#   str_decoded = str_decoded.replace(chr(ord('z') + 1), '')
#   str_decoded = str_decoded.replace(chr(ord('a') - 1), ' ')

#   print 'Original:\n%s' % original
#   print 'Decoded:\n%s' %str_decoded


Conv1 (?, 22, 200, 64)
BN1 (?, 22, 200, 64)
Pool1 (?, 11, 100, 64)
Conv2 (?, 11, 100, 128)
BN2 (?, 11, 100, 128)
Pool2 (?, 5, 50, 128)
Conv3 (?, 5, 50, 256)
BN3 (?, 5, 50, 256)
Pool3 (?, 2, 25, 256)
Conv4 (?, 2, 25, 512)
BN4 (?, 2, 25, 512)
Pool4 (?, 1, 12, 512)
CNN features (?, 12, 512)
RNN_seq (12, ?, 512)
Tensor("CTCLoss:0", shape=(?,), dtype=float32)
Tensor("Mean:0", shape=(), dtype=float32)


TypeError: ignored